In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from chess import Board
import tensorflow as tf


# inspired by Github user Skripkon 
# https://github.com/Skripkon/chess-engine/blob/main/engines/tensorflow/train_and_predict.ipynb
# this ran 100 games for each loaded model against the Stockfish bot and saved the results 

In [3]:
# Load in the model that you want to use
# and the move encoding

model = tf.keras.models.load_model("models/simulated_filtered_model/DENSE_50EPOCHS.keras")
import pickle

with open("models/simulated_filtered_model/DENSE_move_to_int.pkl", "rb") as f:
    move_to_int = pickle.load(f)

with open("models/simulated_filtered_model/DENSE_int_to_move.pkl", "rb") as f:
    int_to_move = pickle.load(f)


In [4]:
#translating the board into a matrix for the predition process 
def board_to_matrix(board: Board):
    matrix = np.zeros((8, 8, 12))
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[row, col, piece_type + piece_color] = 1
    return matrix

In [5]:
import chess
import chess.engine
import chess.pgn
import random
from tqdm import tqdm

# stockfish engine path
# this is the path on my Home machine it should be chnaged if you are using it 
STOCKFISH_PATH = "/opt/homebrew/bin/stockfish"
stockfish_engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)

# engine stats
stockfish_engine.configure({
    "UCI_LimitStrength": True,
    "UCI_Elo": 2000
})

# stockfish model move
def stockfish_move(board):
    result = stockfish_engine.play(board, chess.engine.Limit(time=0.1))
    return result.move.uci()

# our model move
def predict_next_move(board):
    board_matrix = board_to_matrix(board).reshape(1, 8, 8, 12)
    predictions = model.predict(board_matrix, verbose = 0)[0]
    legal_moves = list(board.legal_moves)
    legal_moves_uci = [move.uci() for move in legal_moves]
    sorted_indices = np.argsort(predictions)[::-1]
    for move_index in sorted_indices:
        move = int_to_move[move_index]
        if move in legal_moves_uci:
            return move
    return None

# play n games and write it to a pgn filem file name must be chnaged accordingly
with open("newmodelsim.pgn", "w") as pgn_file: # change accordingly
    # counter to see how many is done
    for game_num in tqdm(range(100)):
        # make a game 
        board = chess.Board()
        game = chess.pgn.Game()

        # write the game headers
        game.headers["White"] = "newmodel" 
        game.headers["Black"] = "Stockfish_2000"

        node = game

        # play the game
        while not board.is_game_over():
            if board.turn == chess.WHITE:
                move_uci = predict_next_move(board)
            else:
                move_uci = stockfish_move(board)

            move = chess.Move.from_uci(move_uci)
            board.push(move)
            node = node.add_variation(move)

        # save the outcome 
        outcome = board.outcome()
        if outcome:
            game.headers["Result"] = outcome.result()
        else:
            game.headers["Result"] = "*"

        # write to file 
        print(game, file=pgn_file, end="\n\n")

stockfish_engine.quit()

100%|██████████| 100/100 [04:19<00:00,  2.60s/it]
